# Retrieval

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

### Document Loader

In [3]:
# WebBaseLoader: 웹 페이지 내용 읽어오기 (크롤링)
from langchain_community.document_loaders import WebBaseLoader

url = "https://ko.wikipedia.org/wiki/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC:%EC%A0%95%EC%B1%85%EA%B3%BC_%EC%A7%80%EC%B9%A8"

loader = WebBaseLoader(url)
documents = loader.load()
# print(documents[0].metadata['title'])
print(documents[0].page_content[:1000])

USER_AGENT environment variable not set, consider setting it to identify your requests.






위키백과:정책과 지침 - 위키백과, 우리 모두의 백과사전



























본문으로 이동







주 메뉴





주 메뉴
사이드바로 이동
숨기기



		둘러보기
	


대문최근 바뀜요즘 화제임의의 문서로특수 문서 목록





		사용자 모임
	


사랑방사용자 모임관리 요청





		편집 안내
	


소개도움말정책과 지침질문방



















검색











검색






















보이기
















기부

계정 만들기

로그인








개인 도구





기부 계정 만들기 로그인





		로그아웃한 편집자를 위한 문서 더 알아보기



기여토론




























목차
사이드바로 이동
숨기기




처음 위치





1
최상위 정책








2
'정책과 지침'이란?








3
준수








4
집행








5
문서 내용








6
정책과 지침은 백과사전의 일부가 아닙니다








7
채택 과정




채택 과정 하위섹션 토글하기





7.1
제안과 채택








7.2
내용 변경






7.2.1
실질적인 변경










7.3
격하










8
같이 보기








9
외부 링크


















목차 토글







위키백과:정책과 지침



110개 언어




Afrikaansአማርኛالعربيةالدارجةمصرىঅসমীয়াАварAzərbaycancaتۆرکجهБашҡортсаBoarischБеларускаяБеларуская (тарашкевіца)БългарскиभोजपुरीBanjarবাংলাBrezhonegBosanskiCatalàНохчийнکوردیČeštinaCymraegDanskDeutschΕλληνικάEnglishEsperantoEspañolEuskaraفارسیSuomiFrançaisGa

In [4]:
!pip install pypdf

In [5]:
# PyPDFLoader : PDF 문서를 로드해서 텍스트 읽어오기
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("./data/The_Adventures_of_Tom_Sawyer.pdf")
documents = loader.load()
print(len(documents))
print(documents[0].metadata)
print('=' * 100)
print(documents[2].page_content)

35
{'producer': '3-Heights(TM) PDF Optimization Shell 5.9.1.5 (http://www.pdf-tools.com)', 'creator': 'Acrobat PDFMaker 7.0 dla programu Word', 'creationdate': '2006-08-26T00:50:00+02:00', 'author': 'GOLDEN', 'company': 'c', 'title': 'Microsoft Word - 1', 'moddate': '2021-01-27T15:00:11+01:00', 'source': './data/The_Adventures_of_Tom_Sawyer.pdf', 'total_pages': 35, 'page': 0, 'page_label': '1'}
The Adventures of                 
Tom Sawyer 
 
MARK TWAIN 
Level 1 
 
Retold by Jacqueline Kehl                                                    
Series Editors: Andy Hopkins and Jocelyn Potter


### Embedding Model

In [6]:
from langchain_openai import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

text = "The quick brown fox jumps over the lazy dog."
vector = embedding_model.embed_query(text)
print(len(vector))

1536


In [7]:
docs = [document.page_content for document in documents]
vects = embedding_model.embed_documents(docs)

print(len(vects))
print(len(vects[0]))

35
1536


### Vector Store

In [8]:
from langchain.vectorstores import FAISS

txt_embeddings = list(zip(docs, vects))
vector_store = FAISS.from_embeddings(txt_embeddings, embedding_model)
# vector_store = FAISS.from_documents(documents, embedding_model)

In [9]:
vector_store.similarity_search("Tom Sawyer", k=3)

[Document(id='16760b21-bf8d-4f5a-9768-98334b97592f', metadata={}, page_content='Introduction \n \n \nOne Saturday afternoon Tom wanted to have an adventure                    \nbecause he didn’t want to think about Injun Joe. He went \nto Huck and said, “I’m going to look for treasure. Do you \nwant to come with me?” \n \nTom Sawyer loves adventures. He has a lot of adventures \nat home, at school, and with his friends. He has one \nadventure in a cave. But why is he there? What does he \nsee in the cave? And why is he afraid? \n \nMark Twain (1835-1910) is a famous American writer. \nHis name was Samuel Clemens. Young Samuel lived in \nHannibal, Missouri, a small town on the Mississippi River. \nHe loved the river and he liked watching the big boats                  \non it. \nSamuel loved adventures. He worked on boats on the \nMississippi River for two years. Then he went to Nevada. \nHe looked for treasure, but he didn’t find it. He worked for \na newspaper there. His stories were 

### Retriever

In [10]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

retriever = vector_store.as_retriever()
print(retriever)

retrieval_qa = RetrievalQA.from_chain_type(
    llm=model,
    retriever=retriever,
    chain_type='stuff'
)

tags=['FAISS', 'OpenAIEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002255263C400> search_kwargs={}


In [11]:
response = retrieval_qa.invoke('마을 무덤에 있던 남자를 누가 죽였나요?')

In [12]:
response

{'query': '마을 무덤에 있던 남자를 누가 죽였나요?',
 'result': '의사(doctor)를 죽인 것은 인준 조(Injun Joe)입니다.'}